In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [8]:
X_full = pd.read_csv('train.csv', index_col='Id')

In [9]:
X_test_full = pd.read_csv('test.csv', index_col='Id')

In [10]:
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True) #drop missing values in the target dataframe
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

In [11]:
X = X_full.select_dtypes(exclude=['object'])
X_test= X_test_full.select_dtypes(exclude=['object'])

X_train,y_train,X_valid,y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
Id,,,,,,,,,,,,,,,,,,,,,
619,20,90.0,11694,9,5,2007,2007,452.0,48,0,...,774,0,108,0,0,260,0,0,7,2007
871,20,60.0,6600,5,5,1962,1962,0.0,0,0,...,308,0,0,0,0,0,0,0,8,2009
93,30,80.0,13360,5,7,1921,2006,0.0,713,0,...,432,0,0,44,0,0,0,0,8,2009
818,20,NaN,13265,8,5,2002,2002,148.0,1218,0,...,857,150,59,0,0,0,0,0,7,2008
303,20,118.0,13704,7,5,2001,2002,150.0,0,0,...,843,468,81,0,0,0,0,0,1,2006


In [19]:
print(X_train.shape )#shape of teh feature training set
print(f"num of rows:{X_train.shape[0]}")
print(f"num of columns:{X_train.shape[1]}")

(1168, 36)
num of rows:1168
num of columns:36


In [17]:
missing_val_count_by_column = (X_train.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0].sum())

276
